Let's consider a hypothetical bons with a par value of 100, that pays 6% coupon semi-annually issued on January 15th, 2015 and set to mature on January 15th, 2016. The bond will pay a coupon on July 15th, 2015 and January 15th, 2016. The par amount of 100 will also be paid on the January 15th, 2016.

To make things simpler, lets assume that we know the spot rate of the treasury as of January 15th, 2015. The annualized spot rates are 0.5% for 6 months and 0.7% for 1 year point. Lets calculate the fair value of this bond.

In [10]:
PV_Jul = 3/pow(1+0.005, 1/2)
PV_Jan = (100+3)/pow(1+0.007, 1)
FV = PV_Jul+PV_Jan
FV

105.27653992490681

Lets calculate the same thing using QuantLib

In [11]:
import QuantLib as ql

In [14]:
todaysDate = ql.Date(15, 1, 2015)
ql.Settings.instance().evaluationDate = todaysDate
spotDates = [ql.Date(15, 1, 2015), ql.Date(15, 6, 2015), ql.Date(15, 1, 2016)]
spotRates = [0.0, 0.005, 0.007]
dayCount = ql.Thirty360(ql.Thirty360.BondBasis)
calendar = ql.UnitedStates(ql.UnitedStates.NYSE)
interpolation = ql.Linear()
compounding = ql.Compounded
compoundingFrequency = ql.Annual

In [16]:
# construct a term structure of spot rates (used to price bonds and other interest rate derivatives)
spotCurve = ql.ZeroCurve(spotDates, spotRates, dayCount, calendar, interpolation, compounding, compoundingFrequency)
spotCurveHandle = ql.YieldTermStructureHandle(spotCurve)

So far we have created the term structure and the variables are rather self explanatory. Now lets contruct the fixed rate bond. 

In [18]:
issueDate = ql.Date(15, 1, 2015)
maturityDate = ql.Date(15, 1, 2016)
tenor = ql.Period(ql.Semiannual)
calendar = ql.UnitedStates(ql.UnitedStates.NYSE)        
businessConvention = ql.Unadjusted                    
dateGeneration = ql.DateGeneration.Backward
monthEnd = False


In [19]:
schedule = ql.Schedule(issueDate, maturityDate, tenor, calendar, businessConvention, businessConvention, dateGeneration, monthEnd)
list(schedule)

[Date(15,1,2015), Date(15,7,2015), Date(15,1,2016)]

In [21]:
# now lets build the coupon
dayCount = ql.Thirty360(ql.Thirty360.BondBasis)
couponRate = .60
coupons = [couponRate]

In [22]:
settlementDays = 0
faceValue = 100
fixedRateBond = ql.FixedRateBond(settlementDays, faceValue, schedule, coupons, dayCount)

In [28]:
# create a bond engine with the term structure as input 
# set the bond to use this bond engine
bondEngine = ql.DiscountingBondEngine(spotCurveHandle)
fixedRateBond.setPricingEngine(bondEngine)

In [29]:
fixedRateBond.NPV() # Net Present Value

159.01735656074857